# 주소 정리

## library

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from tqdm import tqdm

## 데이터 로드

In [28]:
df = pd.read_csv('2020-22년_급속충전.csv', index_col=0, usecols=['주소', '지역', '시군구'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11665822 entries, 강원도 to 서울특별시
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   시군구     object
 1   주소      object
dtypes: object(2)
memory usage: 267.0+ MB


In [29]:
df = df.reset_index()
df.head()

,지역,시군구,주소
0,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향)
1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향)
2,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향)
3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향)
4,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9


In [262]:
adr_uni = df['주소'].unique()
len(adr_uni)

4364

In [263]:
df2 = df.drop_duplicates(['주소']).reset_index(drop=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4364 entries, 0 to 4363
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      4364 non-null   object
 1   시군구     4364 non-null   object
 2   주소      4364 non-null   object
dtypes: object(3)
memory usage: 102.4+ KB


In [264]:
adr = df2['주소'].to_numpy()
len(adr)

4364

### 특이주소 처리

In [265]:
# 경상북도 경주시 천원2길 9 경북 경주시 교동 153-5
# 충청북도 단양군 영춘면 온달평강3로 9 온달평강3로 9-1
# 광주광역시 북구 첨단 벤처로 108번길 9

In [266]:
count = 0
for i, one in enumerate(adr):
    if '경상북도 경주시 천원2길 9 경북 경주시 교동 153-5' in one:
        adr[i] = '경상북도 경주시 교동 천원2길 9'
        count += 1
count

1

In [267]:
count = 0
for i, one in enumerate(adr):
    if '충청북도 단양군 영춘면 온달평강3로 9 온달평강3로 9-1' in one:
        adr[i] = '충청북도 단양군 영춘면 온달평강3로 9'
        count += 1
count

1

In [268]:
count = 0
for i, one in enumerate(adr):
    if '광주광역시 북구 첨단 벤처로 108번길 9' in one:
        adr[i] = '광주광역시 북구 첨단벤처로108번길 9'
        count += 1
count

1

### 빈칸 2개 -> 1개

In [269]:
count = 0
for i, one in enumerate(adr):
    if '  ' in one:
        adr[i] = one.replace('  ', ' ')
        count += 1
count

128

### 컴마(,) 제거

In [270]:
count = 0
for i, one in enumerate(adr):
    if ',' in one:
        adr[i] = one.replace(',', '')
        count += 1
count

324

### 복수 주소 오류
- 한주소에 2개 주소 들어있는 오류
- 동일지역 : 56
- 다른지역 : 2

#### 동일지역

In [271]:
np.where(adr == '전라남도 무안군 망운면 운해로 1436-10 전라남도 함평군 신광면 학동로 1398-9')

(array([2608], dtype=int64),)

In [272]:
adr[2608]

'전라남도 무안군 망운면 운해로 1436-10 전라남도 함평군 신광면 학동로 1398-9'

In [273]:
si = df2['시군구'].to_numpy()

In [274]:
count = 0
for i, one in tqdm(enumerate(adr)):
    first = list(one.split())[0]
    one = one.replace(' ' + first, '$' + first)
    multi_adr = list(one.split('$'))
    if len(multi_adr) == 2:
        count += 1
        for j, two in enumerate(multi_adr):
            if si[i] in two:
                adr[i] = multi_adr[j]
count

4364it [00:00, 312882.56it/s]


56

In [275]:
adr[2608]

'전라남도 무안군 망운면 운해로 1436-10'

#### 다른지역

In [276]:
sep = ' 광주광역시| 전라남도| 전라북도| 제주특별자치도| 세종특별자치시| 서울특별시| 대전광역시| 경기도| 인천광역시| 경상남도| 강원도| 충청북도| 경상북도| 충청남도| 울산광역시| 대구광역시| 부산광역시'

In [277]:
count = 0
for i, one in tqdm(enumerate(adr)):
    multi_local = list(re.split(sep, one))
    if len(multi_local) == 2:
        count += 1
count

4364it [00:00, 312727.54it/s]


2

In [278]:
rep_dict = {" 광주광역시":"$광주광역시" ," 전라남도":"$전라남도"," 전라북도":"$전라북도"," 제주특별자치도":"$제주특별자치도"," 세종특별자치시":"$세종특별자치시"," 서울특별시":"$서울특별시"," 대전광역시":"$대전광역시"," 경기도":"$경기도"," 인천광역시":"$인천광역시"," 경상남도":"$경상남도"," 강원도":"$강원도"," 충청북도":"$충청북도"," 경상북도":"$경상북도"," 충청남도":"$충청남도"," 울산광역시":"$울산광역시"," 대구광역시":"$대구광역시"," 부산광역시":"$부산광역시"}

In [279]:
np.where(adr == '전라북도 순창군 풍산면 반월리 12-4 충청남도 공주시 탄천면 삼각리 523-2')

(array([2456], dtype=int64),)

In [280]:
adr[2456]

'전라북도 순창군 풍산면 반월리 12-4 충청남도 공주시 탄천면 삼각리 523-2'

In [281]:
count = 0
for i, one in tqdm(enumerate(adr)):
    for k, v in rep_dict.items():
        one = one.replace(k, v)
    multi_adr = list(one.split('$'))
    if len(multi_adr) == 2:
        count += 1
        for j, two in enumerate(multi_adr):
            if si[i] in two:
                adr[i] = multi_adr[j]
count

4364it [00:00, 160837.43it/s]


2

In [282]:
adr[2456]

'전라북도 순창군 풍산면 반월리 12-4'

In [283]:
adr[44]

'서울특별시 성동구 광나루로4가길 23'

### 도로명 주소 정리
- [길 숫자] 분리
- [로 숫자] 분리
- -문자 (숫자)(번)길 붙이기
    - [로 숫자(번)길] 붙이기
    - [앙 (숫자)길] 붙이기
    - [장 (숫자)길] 붙이기
    - [솜 (숫자)로] 붙이기
    - [길 (숫자)번길] 붙이기
    - [터 (숫자)길] 붙이기
    - [개 (숫자)길] 붙이기

#### [분리] -길 숫자
- 주의
    - 743, 서울특별시 영등포구 신길1동 116-15
    - 3673, 경상북도 포항시 남구 구룡포읍 구룡포길117번길 28-8

In [284]:
np.where(adr == '전라북도 전주시 완산구 신촌3길24번지')

(array([15], dtype=int64),)

In [285]:
adr[15]

'전라북도 전주시 완산구 신촌3길24번지'

In [286]:
s = adr[15]
split1 = re.compile('([가-힣0-9]+길)([0-9]+)')
split1.sub('\g<1> \g<2>', s)

'전라북도 전주시 완산구 신촌3길 24번지'

In [287]:
adr[743]

'서울특별시 영등포구 신길1동 116-15'

In [288]:
s = adr[743]
split1.sub('\g<1> \g<2>', s)

'서울특별시 영등포구 신길 1동 116-15'

In [289]:
adr[3673]

'경상북도 포항시 남구 구룡포읍 구룡포길117번길 28-8'

In [290]:
s = adr[3673]
split1.sub('\g<1> \g<2>', s)

'경상북도 포항시 남구 구룡포읍 구룡포길 117번길 28-8'

In [291]:
count = 0
idx = []
for i, one in enumerate(adr):
    if split1.search(one):
        print(i, one)
        idx.append(i)
        count += 1
count

15 전라북도 전주시 완산구 신촌3길24번지
120 충청북도 제천시 의림대로42길34
547 서울특별시 송파구 오금로68길8
559 제주특별자치도 제주시 한림읍 금악서길133
717 서울특별시 송파구 성내천로33가길20
743 서울특별시 영등포구 신길1동 116-15
834 경상남도 거창군 거창읍 심소정길39-36
887 경기도 파주시 탄현면 헤이리마을길82-105
1000 제주특별자치도 제주시 조천읍 선흘서5길7
1201 강원도 삼척시 수로부인길453
1332 충청북도 청주시 흥덕구 직지대로620번길70
1342 충청북도 괴산군 청안면 청안읍내로2길10
1479 경기도 양평군 개군면 하자포길33번길 8
1570 경상북도 포항시 북구 흥해읍 초곡길36번길 30
1675 충청북도 괴산군 불정면 목도로2길11
1819 울산광역시 중구 단장골길1
1854 강원도 철원군 동송읍 이평2로17번길32
1919 강원도 삼척시 근덕면 상맹방길30-80
1951 충청북도 괴산군 장연면 미선로오가5길7
1954 경상남도 함양군 안의면 농월정길9-13
2058 울산광역시 울주군 웅촌면 새초천길12
2162 서울특별시 강서구 화곡로65길62
2586 경상북도 의성군 비안면 강변길59
2694 경상북도 포항시 장기면 양포항길12
2747 부산광역시 강서구 공항진입로42번길20
2780 제주특별자치도 제주시 용두암길15
2851 강원도 태백시 보드미길8(장성동)
2970 경상남도 합천군 합천읍 교동1길33
2976 울산광역시 울주군 삼동면 보삼길550
2979 충청북도 괴산군 칠성면 명태재로외사4길1
2990 강원도 강릉시 성산면 삼포암길133
2999 경상남도 창녕군 길곡면 증산2길19
3008 경상북도 칠곡군 동명면 남원로5길77
3017 충청남도 부여군 양화면 입포로53번길6
3025 경상남도 김해시 삼안로 112번길20-61
3030 경상북도 봉화군 봉성면 부랭이길88
3042 인천광역시 중구 신포로27번길80
3043 전라남도 보성군 용문길18
3059 전라남도 보성군 미력면

71

In [292]:
len(idx)

71

In [66]:
# 15 전라북도 전주시 완산구 신촌3길24번지
# 120 충청북도 제천시 의림대로42길34
# 547 서울특별시 송파구 오금로68길8
# 559 제주특별자치도 제주시 한림읍 금악서길133
# 717 서울특별시 송파구 성내천로33가길20

# 834 경상남도 거창군 거창읍 심소정길39-36
# 887 경기도 파주시 탄현면 헤이리마을길82-105
# 1000 제주특별자치도 제주시 조천읍 선흘서5길7
# 1201 강원도 삼척시 수로부인길453
# 1332 충청북도 청주시 흥덕구 직지대로620번길70
# 1342 충청북도 괴산군 청안면 청안읍내로2길10


# 1675 충청북도 괴산군 불정면 목도로2길11
# 1819 울산광역시 중구 단장골길1
# 1854 강원도 철원군 동송읍 이평2로17번길32
# 1919 강원도 삼척시 근덕면 상맹방길30-80
# 1951 충청북도 괴산군 장연면 미선로오가5길7
# 1954 경상남도 함양군 안의면 농월정길9-13
# 2058 울산광역시 울주군 웅촌면 새초천길12
# 2162 서울특별시 강서구 화곡로65길62
# 2586 경상북도 의성군 비안면 강변길59
# 2694 경상북도 포항시 장기면 양포항길12
# 2747 부산광역시 강서구 공항진입로42번길20
# 2780 제주특별자치도 제주시 용두암길15
# 2851 강원도 태백시 보드미길8(장성동)
# 2970 경상남도 합천군 합천읍 교동1길33
# 2976 울산광역시 울주군 삼동면 보삼길550
# 2979 충청북도 괴산군 칠성면 명태재로외사4길1
# 2990 강원도 강릉시 성산면 삼포암길133
# 2999 경상남도 창녕군 길곡면 증산2길19
# 3008 경상북도 칠곡군 동명면 남원로5길77
# 3017 충청남도 부여군 양화면 입포로53번길6
# 3025 경상남도 김해시 삼안로 112번길20-61
# 3030 경상북도 봉화군 봉성면 부랭이길88
# 3042 인천광역시 중구 신포로27번길80
# 3043 전라남도 보성군 용문길18
# 3059 전라남도 보성군 미력면 가평길44
# 3061 경상북도 영양군 영양읍 군청길37
# 3069 경상남도 창원시 마산합포구 현동7길27
# 3073 대전광역시 유성구 송림로 19번길35
# 3078 대구광역시 동구 첨단로8길32
# 3079 충청남도 논산시 산양1길2
# 3088 경상남도 함양군 함양읍 삼휴길60-30
# 3101 대전광역시 유성구 노은동로 87번길89
# 3152 충청북도 영동군 황간면 원촌동1길54-8
# 3165 경상북도 봉화군 춘양면 의양로5길19
# 3174 경상북도 청송군 현서면 달정길9
# 3177 경상북도 청송군 주왕산면 이전앞길27
# 3189 경상북도 영양군 영양읍 군민회관길46
# 3192 경상북도 청송군 진보면 이촌1길23-1
# 3197 충청북도 제천시 내토로16안길8-1
# 3211 충청북도 영동군 용화면 휴양림길60
# 3224 경기도 시흥시 시흥대로268번길8
# 3272 강원도 춘천시 칠전동길28
# 3273 전라북도 군산시 창길21
# 3324 전라북도 군산시 미장8길7
# 3376 경상북도 영주시 중앙로63번길41-11
# 3387 전라남도 목포시 남악로162번길25
# 3396 울산광역시 울주군 청량읍 처용산업4길51
# 3399 전라남도 보성군 회천면 우암길21
# 3401 충청남도 금산군 금성면 의총길25
# 3413 경상북도 구미시 광평길55
# 3415 전라북도 익산시 익산대로 52길27
# 3419 경상남도 창원시 의창구 태복산로15번길8
# 3424 경상남도 진주시 가좌길29번길63
# 3438 경상북도 칠곡군 왜관읍 평장3길47
# 3550 경상북도 안동시 학가산온천길42
# 3673 경상북도 포항시 남구 구룡포읍 구룡포길117번길 28-8
# 3742 경상남도 창원시 의창구 사림로 99번길63

'전라북도 전주시 완산구 신촌3길 24번지'

In [259]:
adr[15]

'전라북도 전주시 완산구 신촌3길24번지'

In [260]:
adr[15] = adr[15].replace('24번지', ' 24번지')
adr[15]

'전라북도 전주시 완산구 신촌3길 24번지'

In [261]:
adr[120]

'충청북도 제천시 의림대로42길34'

#### [분리] -로 숫자

In [53]:
np.where(adr == '경기도 이천시 마장면 중부고속도로82')

(array([85], dtype=int64),)

In [54]:
adr[85]

'경기도 이천시 마장면 중부고속도로82'

In [57]:
np.where(adr == '부산광역시 중구 중앙대로2 (중앙동7가)')

(array([1062], dtype=int64),)

In [58]:
adr[1062]

'부산광역시 중구 중앙대로2 (중앙동7가)'

In [69]:
s = adr[85]
split2 = re.compile('([가-힣0-9]+로)([0-9]+)')
split2.sub('\g<1> \g<2>', s)

'경기도 이천시 마장면 중부고속도로 82'

In [70]:
s = adr[1062]
split2.sub('\g<1> \g<2>', s)

'부산광역시 중구 중앙대로 2 (중앙동7가)'

In [71]:
count = 0
for i, one in enumerate(adr):
    if split2.search(one):
        adr[i] = split2.sub('\g<1> \g<2>', one)
        count += 1
count

366

In [72]:
adr[85]

'경기도 이천시 마장면 중부고속도로 82'

In [73]:
adr[1062]

'부산광역시 중구 중앙대로 2 (중앙동7가)'

#### [붙이기] -문자 (숫자)(번)길|로

In [107]:
# 인천광역시 서구 청라 커낼로
# 대전광역시 유성구 테크노 9로
# 제주특별자치도 제주시 산천단 동3길
# 경기도 하남시 미사강변 한강로
# 경상북도 상주시 영남 제일로
# 경기도 이천시 호법면 프리미엄 아울렛로
# 충청남도 보령시 갓바위 1길
# 울산광역시 북구 진장 유통로
# 부산광역시 강서구 명지오션시티 13로

In [61]:
np.where(adr == '서울특별시 마포구 상암동 월드컵북로 58길 15 지하4층 18')

(array([1392], dtype=int64),)

In [62]:
adr[1392]

'서울특별시 마포구 상암동 월드컵북로 58길 15 지하4층 18'

In [80]:
for i, one in enumerate(adr):
    if '대전광역시 유성구 테크노 9로' in one:
        print(i)

1041


In [81]:
adr[1041]

'대전광역시 유성구 테크노 9로 35 입구 국기계양대 왼쪽'

In [87]:
s = adr[1041]
split3 = re.compile('([가-힣]+(?![가-힣]+시 |[가-힣]+군 |[가-힣]+구 |[가-힣]+도 |[가-힣]+읍 |[가-힣]+면 |[가-힣]+동 |[가-힣]+리 )) ([0-9가-힣]*[번]*[길|로] )')
if split3.search(s):
    print(split3.sub('\g<1>\g<2>', s))

대전광역시 유성구 테크노9로 35 입구 국기계양대 왼쪽


In [84]:
for i, one in enumerate(adr):
    if '인천광역시 서구 청라 커낼로' in one:
        print(i)

901


In [94]:
adr[901]

'인천광역시 서구 청라 커낼로 252(경서동)'

In [97]:
s = adr[901]
if split3.search(s):
    print(split3.sub('\g<1>\g<2>', s))

인천광역시 서구 청라커낼로 252(경서동)


In [89]:
for i, one in enumerate(adr):
    if '구로구' in one:
        print(i)

538
617
792
2023
2024
4206


In [90]:
adr[538]

'서울특별시 구로구 개봉동 134-8'

In [99]:
s = adr[538]
split3.search(s)

In [110]:
s = adr[538]
p = re.compile('[가-힣]+로 ')
p.search(s)

In [100]:
for i, one in enumerate(adr):
    if '경기도 하남시 미사강변 한강로' in one:
        print(i)

1034


In [101]:
adr[1034]

'경기도 하남시 미사강변 한강로 299 지하주차장 입구 앞'

In [103]:
s = adr[1034]
if split3.search(s):
    print(split3.sub('\g<1>\g<2>', s))

경기도 하남시 미사강변한강로 299 지하주차장 입구 앞


In [104]:
for i, one in enumerate(adr):
    if '충청남도 보령시 갓바위 1길' in one:
        print(i)

2497


In [105]:
adr[2497]

'충청남도 보령시 갓바위 1길 10-30'

In [106]:
s = adr[2497]
if split3.search(s):
    print(split3.sub('\g<1>\g<2>', s))

충청남도 보령시 갓바위1길 10-30


In [111]:
count = 0
for i, one in enumerate(adr):
    if split3.search(one):
        adr[i] = split3.sub('\g<1>\g<2>', one)
        count += 1
count

2817

In [112]:
adr[[1392, 1041, 901, 1034, 2497]]

array(['서울특별시 마포구 상암동월드컵북로 58길 15 지하4층 18',
       '대전광역시 유성구 테크노9로 35 입구 국기계양대 왼쪽', '인천광역시 서구 청라커낼로 252(경서동)',
       '경기도 하남시 미사강변한강로 299 지하주차장 입구 앞', '충청남도 보령시 갓바위1길 10-30'],
      dtype=object)

In [113]:
adr[538]

'서울특별시 구로구 개봉동 134-8'

### 구주소 정리
- [분리] "-동|면|읍" "숫자"
- [붙이기] "-동" "숫자가" 붙이기

#### [분리] "-동|면|읍" "숫자"

In [198]:
# 16, 163, 1206, 1377, 1568, 1647, 1811, 2396
split5 = re.compile('([가-힣]+[동|면|읍])([0-9]+)')
for i, one in enumerate(adr):
    if split5.search(one):
        print(i, one)

16 제주특별자치도 제주시 연동2334-4
163 경상남도 창원시 성산구 사파동64-2
283 전라북도 전주시 덕진구 덕진동2가 179-6
389 충청북도 단양군 적성면기동2길 32-37 (기동리)
439 전라북도 전주시 완산구 효자동3가 1712-10
443 전라북도 전주시 완산구 효자동2가 1234-3 5층
601 전라북도 남원시 아영면88올림픽고속도로 75 (아곡리)
602 전라북도 남원시 아영면88올림픽고속도로 76 (아곡리)
603 전라북도 순창군 순창읍88올림픽고속도로 22 (백산리)
605 전라북도 전주시 덕진구 인후동1가 807-6
607 전라북도 전주시 덕진동2가 694-1
645 경상북도 구미시인동14길 14
649 대구광역시 북구 노원동2가 214
687 경상남도 창원시 마산합포구신마산시장길 33 (창포동3가)
854 경상남도 창원시 진해구용재로 46  웅동2동 주민센터 주차장 내
982 서울특별시 종로구인사동5길 29 지상주차장
1056 경상남도 창원시 장군동4가 3-6
1062 부산광역시 중구중앙대로 2 (중앙동7가)
1135 충청북도 청주시 서원구 현도면선동1길 9
1139 충청남도 당진시무수동2길 25-21
1206 대전광역시 동구 용운동423-22
1305 제주특별자치도 제주시 조천읍516로 1865 성판악휴게소
1357 제주특별자치도 제주시 산천단동3길 2 북쪽주차장 입구
1368 울산광역시 울주군 언양읍언동3길 20-48
1377 대전광역시 동구 홍도동92-3
1426 강원도 삼척시당저동1길 17-9
1568 경상북도 구미시 오태동659-13
1647 강원도 태백시 소도동326
1748 인천광역시 중구 북성동1가 97-32
1811 경상남도 창원시 진해구 명동580 좌측 주차장
2396 울산광역시 남구 선암동584-8번지
2421 전라북도 전주시 덕진구 송천동2가 1257
2423 전라북도 전주시 덕진구 송천동2가 1316
2616 경상남도 창원시 마산회원구석전동4길 52
2854 인천광역시 중구 송월동2가 4
2862 전라북도

In [199]:
adr[[16, 163, 1206, 1377, 1568, 1647, 1811, 2396]]

array(['제주특별자치도 제주시 연동2334-4', '경상남도 창원시 성산구 사파동64-2',
       '대전광역시 동구 용운동423-22', '대전광역시 동구 홍도동92-3', '경상북도 구미시 오태동659-13',
       '강원도 태백시 소도동326', '경상남도 창원시 진해구 명동580 좌측 주차장',
       '울산광역시 남구 선암동584-8번지'], dtype=object)

In [206]:
s = '충청북도 청주시 서원구 현도면선동2길 232-9'
p = re.compile('([가-힣]+동)([0-9]*(?![0-9길로]))')
p.search(s)

In [207]:
s = '강원도 태백시 소도동326'
p = re.compile('([가-힣]+동)([0-9]*(?![0-9길로]))')
p.search(s)

<re.Match object; span=(8, 14), match='소도동326'>

In [211]:
adr[44]

'서울특별시 성동구광나루로 4가길 23'

In [210]:
# 16, 163, 1206, 1377, 1568, 1647, 1811, 2396
split5 = re.compile('([가-힣]+동)([0-9]*(?![0-9길로]))')
for i, one in enumerate(adr):
    if split5.search(one):
        print(i, one)

3 전라남도 구례군 산동면산업로 2319
4 전라남도 구례군 산동면노고단로 1068
11 전라북도 남원시 산내면 덕동리 406
12 전라북도 정읍시 내장동 52
13 전라북도 정읍시 내장동 169-1
16 제주특별자치도 제주시 연동2334-4
20 제주특별자치도 제주시연북로 1(노형동)
24 광주광역시 서구눌재로 420 (세하동)
28 대전광역시 대덕구 신대동 169-1
30 대전광역시 서구 만년동 74
31 대전광역시 서구 복수동 613
32 대전광역시 서구 월평동 1275
33 대전광역시 서구 정림동 127-1
34 대전광역시 중구 사정동 403번지
36 경기도 양주시 옥정동 1039
39 경기도 부천시 상동 549번지
41 경기도 안산시 상록구 이동 620
42 경기도 하남시 망월동 1-1
44 서울특별시 성동구광나루로 4가길 23
45 인천광역시 서구 연희동 187-8
46 인천광역시 남동구 논현동 762-3
47 인천광역시 남동구 서창동 558-3
51 경상남도 거제시 동부면 학동리 264
58 강원도 횡성군 안흥면영동고속도로 153  농축산물판매장 옆
59 강원도 횡성군 안흥면영동고속도로 154  휴게소와 LPG충전소 사이
63 강원도 인제군 인제읍 상동리 430
68 충청북도 영동군 영동읍영동황간로 15
71 충청북도 단양군 적성면 기동리
73 제주특별자치도 제주시 아라동 3008-1
74 제주특별자치도 제주시 아라동 6140-4
83 강원도 강릉시 성산면영동고속도로 231 (보광리)
84 강원도 강릉시 성산면영동고속도로 232 (보광리)
88 경기도 용인시 처인구주북로 94번길 30-1 (고림동)
89 경기도 용인시 처인구주북로 66-14 (고림동)
91 경기도 구리시서울외곽순환고속도로 32 (사노동)
95 경상남도 김해시 대동면 월촌리 770
99 경기도 수원시 권선구동수원로 232 지하2층 주차장 좌측
106 강원도 강릉시종합운동장길 69
113 강원도 횡성군 안흥면영동고속도로 154
114 강원도 횡성군 안흥면영동고속도로 153
121 대전

#### [붙이기] "-동" "(숫자)가" 붙이기

In [114]:
# 전라북도 전주시 덕진구 덕진동 2가 179-6
# 전라북도 전주시 덕진동 2가 694-1

In [115]:
for i, one in enumerate(adr):
    if '전라북도 전주시 덕진구 덕진동 2가 179-6' in one:
        print(i)

283


In [116]:
adr[283]

'전라북도 전주시 덕진구 덕진동 2가 179-6'

In [119]:
split4 = re.compile('([가-힣]+동) ([0-9]+가)')
s = adr[283]
split4.sub('\g<1>\g<2>', s)

'전라북도 전주시 덕진구 덕진동2가 179-6'

In [138]:
for i, one in enumerate(adr):
    if '전라북도 전주시 덕진동' in one:
        print(i)

607


In [139]:
adr[607]

'전라북도 전주시 덕진동 2가 694-1'

In [142]:
for i, one in enumerate(adr):
    if '대구광역시 북구 노원동' in one:
        print(i)

649


In [143]:
adr[649]

'대구광역시 북구 노원동2가 214'

In [144]:
count = 0
for i, one in enumerate(adr):
    if split4.search(one):
        # adr[i] = split4.sub('\g<1>\g<2>', one)
        count += 1
count

2

In [145]:
count = 0
for i, one in enumerate(adr):
    if split4.search(one):
        adr[i] = split4.sub('\g<1>\g<2>', one)
        count += 1
count

2

In [146]:
adr[607]

'전라북도 전주시 덕진동2가 694-1'

In [147]:
adr[649]

'대구광역시 북구 노원동2가 214'

# 코드 마지막